In [2]:
# Utility imports
# General imports
import torch
import torch.nn as nn
import os
# import tensorflow as tf
from scipy import stats
# Used for distributions libraries.
from scipy import stats
# Utility imports
from utils.losses import *
from utils.plotting import *
from utils.training import *

import pickle
import sys
from lightning.pytorch.accelerators import find_usable_cuda_devices


In [3]:
eta = 5e3
lr_vals = [1.0]
F0 = -1.
nu_vals = [1e-6,1e-5]

file_name = 'scan_bce_ECD_results'
loss_funcs = ['linear', 'square', 'exponl']

reps = 100
optimizer = 'ECD'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Data parameters
N = 10**6
X = np.load('data/zenodo/fold/8/X_trn.npy')[:N]
y = np.load('data/zenodo/fold/8/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

class train_val_loader(pl.LightningDataModule):
    def __init__(self, data, N, workers):
        super().__init__()
        self.N = N
        self.data = data
        self.workers = workers
    def prepare_data(self):
        X_train, X_test, y_train, y_test = self.data
        X_train = X_train.astype(np.float32)
        X_test = X_test.astype(np.float32)
        y_train = y_train.astype(np.float32)
        y_test = y_test.astype(np.float32)

        self.X_train = torch.from_numpy(X_train)
        self.X_test = torch.from_numpy(X_test)
        self.y_train = torch.from_numpy(y_train)
        self.y_test = torch.from_numpy(y_test)

        self.train_data = TensorDataset(self.X_train, self.y_train)
        self.test_data = TensorDataset(self.X_test, self.y_test)
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=int(0.1*self.N), shuffle=True, num_workers=self.workers, persistent_workers=True)
    def val_dataloader(self):
        return DataLoader(self.test_data, batch_size=int(0.1*self.N), shuffle=False, num_workers=self.workers, persistent_workers=True)

train_val_data = train_val_loader(data, N, 95)

max_epochs = 500
min_epochs = 15

X_mae = np.load('data/zenodo/fold/8/X_tst.npy')
X_mae = torch.from_numpy(X_mae)
lr_tst = np.load('data/zenodo/fold/8/lr_tst.npy')
lr_tst = torch.from_numpy(lr_tst)

def mae(model_lr):
    abs_dif = abs(model_lr(X_mae) - lr_tst)
    return abs_dif[abs_dif < 100].mean()
def stdae(model_lr):
    abs_dif = abs(model_lr(X_mae) - lr_tst)
    return abs_dif[abs_dif < 100].std()

filestr = 'models/zenodo/mlc/'
dict_list = []
for loss_func in loss_funcs:
    for learning_rate in lr_vals:
        for nu in nu_vals:
            for i in range(reps):
            if loss_func == 'odds': loss_fn = bce; output = 'sigmoid'; lr_fn = odds_lr
            elif loss_func == 'probit': loss_fn = probit_bce; output = 'linear'; lr_fn = probit_lr
            elif loss_func == 'arctan': loss_fn = arctan_bce; output = 'linear'; lr_fn = arctan_lr
    
                params = {'loss_fun':loss_fn, 'd': 6, 'output': output, 'optimizer': optimizer, 'learning_rate': learning_rate, 'eta': eta, 'F0': F0, 'nu': nu}
                model_path = filestr + loss_func + '/ecd/'
    
                checkpoint_callback = ModelCheckpoint(
                    dirpath = model_path,
                    filename = 'model_{}'.format(i),
                    monitor = 'val_loss',
                    mode = 'min',
                    save_weights_only = True
                )
    
                trainer = pl.Trainer(accelerator='cuda', devices=find_usable_cuda_devices(1), max_epochs=max_epochs, callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience = 20), checkpoint_callback], min_epochs=min_epochs, enable_progress_bar=False)
    
                model = create_model_original(**params)
                model.to(device)
                trainer.fit(model, train_val_data)
    
                try: os.mkdir(model_path)
                except OSError as error: print(error)
    
                train_losses = model.train_hist
                val_losses = model.val_hist
    
                checkpoint = torch.load(checkpoint_callback.best_model_path)
                model.load_state_dict(checkpoint['state_dict'])
                model.eval()
                lr = lr_fn(model, m , s)
                mae_1 = mae(lr).detach().numpy()
    
                scan_res = dict(mae = mae_1, optimizer = optimizer, learning_rate = learning_rate, eta = eta, F0 = F0, nu = nu, classifier = loss_func, train_loss = train_losses, val_loss = val_losses, path = checkpoint_callback.best_model_path)
                dict_list.append(scan_res)
                print(f'eta: {eta} lr: {learning_rate} mae: {mae_1} classifier: ', loss_func, f'path: {model_path}')
                del model

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/loca

[Errno 17] File exists: 'models/zenodo/mlc/linear/ecd/'


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | linear1  | Linear  | 448    | train
1 | linear2  | Linear  | 8.3 K  | train
2 | linear3  | Linear  | 8.3 K  | train
3 | linear4  | Linear  | 65     | train
4 | dropout1 | Dropout | 0      | train
5 | dropout3 | Dropout | 0      | train
6 | dropout2 | Dropout | 0      | train
---------------------------------------------
17.1 K    Trainable params
0         Non-trainable params
17.1 K    Total params
0.068     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | linear1  | Linear  | 448    | train
1 | linear2  | Linear  | 8.3 K  | train
2 | linear3  | Linear  | 8.

eta: 5000.0 lr: 1.0 mae: 0.4157039523124695 classifier:  linear path: models/zenodo/mlc/linear/ecd/


`Trainer.fit` stopped: `max_epochs=500` reached.


...

eta: 5000.0 lr: 1.0 mae: 0.4067525267601013 classifier:  exponl path: models/zenodo/mlc/exponl/ecd/


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | linear1  | Linear  | 448    | train
1 | linear2  | Linear  | 8.3 K  | train
2 | linear3  | Linear  | 8.3 K  | train
3 | linear4  | Linear  | 65     | train
4 | dropout1 | Dropout | 0      | train
5 | dropout3 | Dropout | 0      | train
6 | dropout2 | Dropout | 0      | train
---------------------------------------------
17.1 K    Trainable params
0         Non-trainable params
17.1 K    Total params
0.068     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | linear1  | Linear  | 448    | train
1 | linear2  | Linear  | 8.3 K  | train
2 | linear3  | Linear  | 8.3 K  | train
3 | linear4  | Linear  | 65     | train
4 | dropout1 | Dropout | 0      | train
5 | dropout3 | Dropout 

[Errno 17] File exists: 'models/zenodo/mlc/exponl/ecd/'


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | linear1  | Linear  | 448    | train
1 | linear2  | Linear  | 8.3 K  | train
2 | linear3  | Linear  | 8.3 K  | train
3 | linear4  | Linear  | 65     | train
4 | dropout1 | Dropout | 0      | train
5 | dropout3 | Dropout | 0      | train
6 | dropout2 | Dropout | 0      | train
---------------------------------------------
17.1 K    Trainable params
0         Non-trainable params
17.1 K    Total params
0.068     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | linear1  | Linear  | 448    | train
1 | linear2  | Linear  | 8.3 K  | train
2 | linear3  | Linear  | 8.

eta: 5000.0 lr: 1.0 mae: 0.401973158121109 classifier:  exponl path: models/zenodo/mlc/exponl/ecd/


In [ ]:
with open('models/zenodo/bce/' + file_name + '.pkl', 'wb') as fout:
    pickle.dump(results, fout)
    fout.close()